In [ ]:
def Strategy_2():
    import tushare as ts
    import os
    import datetime
    import pymongo
    import pandas

    client = pymongo.MongoClient("127.0.0.1", 27017)
    
    stocks = client.Common.Stock.find({})
    today = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    found = []
    minDays = 1000

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = list(collection.find({}, limit=10, sort=[("date", pymongo.DESCENDING)]))
        if len(res) < 10:
            continue
        days = (today - res[-1]["date"]).days
        if days > minDays:
            continue
        if days < minDays:
            minDays = days
            found = []
        v10 = 0
        v3 = 0
        p5 = 0
        for i in range(10):
            v10 = v10 + res[i]["volume"]
            if i < 3:
                v3 = v3 + res[i]["volume"]
            if i < 5:
                p5 = p5 + res[i]["close"]
        v10 = int(v10 / 10)
        v3 = int(v3 / 3)
        p5 = int(p5 / 5)
        close = res[0]["close"]
        if v3 >= v10 * 2 and close >= p5:
            item = {
                "code": code,
                "name": item["name"],
                "volume_3": v3,
                "volume_10": v10,
                "price_today": close,
                "price_5": p5,
                "volume_factor": (1.0 * v3 / v10),
            }
            found.append(item)


    if len(found) > 0:
        print pandas.DataFrame(found)
    else:
        print("No Result")

In [ ]:
Strategy_2()